# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove the `raise NotImplementedError()` statement.

## Question 1 (2 points)

Complete the function `create_operator` that returns a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$.

The function `create_operator` has

- Input: $\phi$
- Returns: Controlled $U$ gate with the properties described above

Don't create any circuit. Only a gate should be returned.


In [1]:
from cirq import CZPowGate

def create_operator(phi):
    
    # YOUR CODE HERE
    
    CU = CZPowGate(exponent=phi*2)

    
    #raise NotImplementedError()
    
    # Do not modify anything below this line  
    return CU

In [2]:
# You can use this code to test your function by checking the 
# bottom right corner of the unitary matrix
import cirq
from cmath import exp, pi

def test(phi):
    CU = create_operator(phi)
    unitary_matrix = cirq.unitary(CU)
    print(unitary_matrix)

phi=3/5
test(phi)
print(exp(2*pi*phi*1j))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.80901699-0.58778525j]]
(-0.8090169943749475-0.587785252292473j)


In [12]:
# hidden tests in this cell will be used for grading.

In [13]:
# hidden tests in this cell will be used for grading.

## Question 2 (8 points)

Complete the function `estimate_phi` such that given a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$, it estimates and returns $\phi$.

The function `estimate_phi` has

- Input: Controlled $U$ gate
- Returns: Estimate for phi 

You are given iqft and qpe algorithms and you can use them in your solution.


In [3]:
def iqft(n,qubits,circuit):
    
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)
     
    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step
            
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):
    
    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))
    
    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate 
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))
        
        
    #Apply inverse QFT
    iqft(t,control,circuit)
    

In [10]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def estimate_phi(mystery):

    

    #Create cirucit
    circuit = cirq.Circuit()
    
    n=1#Number of qubits in the register storing eigenvector
    t=10
    #mystery=CZPowGate(exponent=phi*2)
    

    #Create t control qubits
    control = [cirq.LineQubit(i) for i in range(t) ]

    #Create n target qubits
    target = [cirq.LineQubit(i) for i in range(t,t+n) ]

    #Set target qubit to state |1> 
    circuit.append(cirq.X.on_each(target))
    
    #Apply QPE
    qpe(t,control, target, circuit, mystery)    
    
    circuit.append(cirq.measure(*control, key='result'))
    
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)
    
    
    #Most frequent observation
    
    freq = list(samples.histogram(key='result').keys())[0]
    #print(freq/2**t)
    
    return freq/2**t

 

In [13]:
# You can use this code to test your function by different operators
import math
def test_qpe(phi):
    operator = CZPowGate(exponent=2*phi)
    return estimate_phi(operator)
#math.isclose( test_qpe(0.23) , 0.23, abs_tol = 1e-2)
assert(math.isclose(test_qpe(0.23),0.23,rel_tol=1e-2))
print(test_qpe(0.23))


0.2470703125


In [20]:
# hidden tests in this cell will be used for grading.

In [ ]:
# hidden tests in this cell will be used for grading.